In [1]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=edccd6dfc00da7433344df17effa354aee003168bbb71a4080477e2e796f0318
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [4]:
import wikipedia
import numpy as np
import nltk
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding
from tensorflow.keras.models import Sequential

In [5]:
nltk.download('wordnet', "/kaggle/working/nltk_data/")
nltk.download('omw-1.4', "/kaggle/working/nltk_data/")
! unzip /kaggle/working/nltk_data/corpora/wordnet.zip -d /kaggle/working/nltk_data/corpora
! unzip /kaggle/working/nltk_data/corpora/omw-1.4.zip -d /kaggle/working/nltk_data/corpora
nltk.data.path.append("/kaggle/working/nltk_data/")

[nltk_data] Downloading package wordnet to
[nltk_data]     /kaggle/working/nltk_data/...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /kaggle/working/nltk_data/...
Archive:  /kaggle/working/nltk_data/corpora/wordnet.zip
   creating: /kaggle/working/nltk_data/corpora/wordnet/
  inflating: /kaggle/working/nltk_data/corpora/wordnet/lexnames  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/data.verb  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/index.adv  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/adv.exc  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/index.verb  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/data.adj  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/index.adj  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/LICENSE  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/citation.bib  
  inflating: /kaggle/working/nltk_data/c

In [6]:
import wikipedia
import numpy as np
import nltk
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding
from tensorflow.keras.models import Sequential


In [7]:
# Download NLTK resources
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [46]:
# Function to fetch data from Wikipedia
def get_wikipedia_data(topic, num_sentences):
    wikipedia.set_lang("en")
    try:
        content = wikipedia.page(topic).content
        sentences = content.split(". ")
        data = []
        for i in range(len(sentences) - num_sentences):
            data.append(" ".join(sentences[i:i+num_sentences]))
        return " ".join(data)
    except wikipedia.exceptions.DisambiguationError as e:
        print(e.options)
        return None
    except wikipedia.exceptions.PageError as e:
        print(e)
        return None

In [47]:
# Define the topics and the number of sentences in each sample
topics = ["Natural_language_processing", "Image_Processing", "Artificial_intelligence"]
num_sentences = 2

data = ""
for topic in topics:
    topic_data = get_wikipedia_data(topic, num_sentences)
    if topic_data:
        data += topic_data


In [48]:
# Preprocess the data
data = re.sub(r'[^\w\s]', '', data.lower())
data = data.replace('\n', '')
data = re.sub(r'\s+', ' ', data)

In [50]:
data

'natural language processing nlp is an interdisciplinary subfield of computer science and information retrieval it is primarily concerned with giving computers the ability to support and manipulate human language it is primarily concerned with giving computers the ability to support and manipulate human language it involves processing natural language datasets such as text corpora or speech corpora using either rulebased or probabilistic ie it involves processing natural language datasets such as text corpora or speech corpora using either rulebased or probabilistic ie statistical and most recently neural networkbased machine learning approaches statistical and most recently neural networkbased machine learning approaches the goal is a computer capable of understanding the contents of documents including the contextual nuances of the language within them the goal is a computer capable of understanding the contents of documents including the contextual nuances of the language within the

In [52]:
# Create character-based sequences
max_sequence_len = 50  # Adjusted max sequence length
step = 1
sentences = []
next_chars = []

In [53]:
# Iterate over the data, considering sequences of length max_sequence_len
for i in range(0, len(data) - max_sequence_len, step):
    # Append the current sequence of max_sequence_len characters to sentences list
    sentences.append(data[i: i + max_sequence_len])
    # Append the next character after the current sequence to next_chars list
    next_chars.append(data[i + max_sequence_len])


In [54]:
# Create a list of unique characters in the data
chars = sorted(list(set(data)))

# Create a dictionary mapping characters to their indices
char_indices = dict((c, i) for i, c in enumerate(chars))

# Create a dictionary mapping indices to characters
indices_char = dict((i, c) for i, c in enumerate(chars))

# Initialize input data (X) and label data (y)
X = np.zeros((len(sentences), max_sequence_len, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)


In [55]:
# Iterate over each sentence in sentences
for i, sentence in enumerate(sentences):
    # Iterate over each character in the current sentence
    for t, char in enumerate(sentence):
        # Set the corresponding character's index in X to 1
        X[i, t, char_indices[char]] = 1
    # Set the corresponding next character's index in y to 1
    y[i, char_indices[next_chars[i]]] = 1


In [56]:
print(len(chars))

43


In [60]:
# Build the SimpleRNN model
model_char = Sequential([
    SimpleRNN(128, input_shape=(max_sequence_len, len(chars)), return_sequences=True),
    SimpleRNN(128),
    Dense(len(chars), activation='softmax')
])

In [61]:
model_char.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [62]:
# Train the model
model_char.fit(X, y, batch_size=256, epochs=80)

Epoch 1/80
935/935 ━━━━━━━━━━━━━━━━━━━━ 16s 13ms/step - accuracy: 0.2650 - loss: 2.5691
Epoch 2/80
935/935 ━━━━━━━━━━━━━━━━━━━━ 18s 11ms/step - accuracy: 0.4397 - loss: 1.9168
Epoch 3/80
935/935 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.5029 - loss: 1.7047
Epoch 4/80
935/935 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.5400 - loss: 1.5698
Epoch 5/80
935/935 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.5660 - loss: 1.4833
Epoch 6/80
935/935 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.5806 - loss: 1.4261
Epoch 7/80
935/935 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.5940 - loss: 1.3770
Epoch 8/80
935/935 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.6037 - loss: 1.3353
Epoch 9/80
935/935 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.6151 - loss: 1.2980
Epoch 10/80
935/935 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.6206 - loss: 1.2722
Epoch 11/80
935/935 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.6260 - loss: 1.2477
Epoch 12/80
935/935 ━━━━━━━━━━

In [63]:
# Function to generate text
def generate_text(seed_text, next_chars, model, max_sequence_len):
    generated = seed_text
    for i in range(next_chars):
        x_pred = np.zeros((1, max_sequence_len, len(chars)))
        for t, char in enumerate(seed_text):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = np.random.choice(len(chars), p=preds)
        next_char = indices_char[next_index]

        seed_text += next_char
        seed_text = seed_text[1:]

        generated += next_char
    return generated


In [87]:
# Generate text
seed_text = "natural language processing is"
next_chars = 20
generated_text = generate_text(seed_text, next_chars, model_char, max_sequence_len)
print(generated_text)

natural language processing isais aooipelimaiaraai


In [68]:
# Generate text
seed_text = "nat"
next_chars = 4
generated_text = generate_text(seed_text, next_chars, model_char, max_sequence_len)
print(generated_text)

nataoaa


In [82]:
# Generate text
seed_text = "nlp"
next_chars = 10
generated_text = generate_text(seed_text, next_chars, model_char, max_sequence_len)
print(generated_text)

nlprriartsaan


In [73]:
# Generate text
seed_text = "s"
next_chars = 4
generated_text = generate_text(seed_text, next_chars, model_char, max_sequence_len)
print(generated_text)

spasa


In [96]:
# Generate text
seed_text = "data"
next_chars = 10
generated_text = generate_text(seed_text, next_chars, model_char, max_sequence_len)
print(generated_text)

dataaasttoaoia


In [97]:
# Generate text
seed_text = "u"
next_chars = 2
generated_text = generate_text(seed_text, next_chars, model_char, max_sequence_len)
print(generated_text)

uas


In [108]:
# Generate text
seed_text = "a"
next_chars = 5
generated_text = generate_text(seed_text, next_chars, model_char, max_sequence_len)
print(generated_text)

asaarr


In [119]:
# Generate text
seed_text = "image"
next_chars = 10
generated_text = generate_text(seed_text, next_chars, model_char, max_sequence_len)
print(generated_text)

imageaa aiaeass
